## Notebook for Generation of Taylor Swift lyrics using fined-tuned distil-GPT-2

- It tests 2 examples of 2 or 3 verses so we can use causal language model distil-GPT-2 to generate the rest of the lyrics

- It uses two strategies Multinomial Sampling (stochastic) and Contrastive Search (deterministic)

- The best results for lyrics were clearly better using Multinomial Sampling (stochastic), than Contrastive Search (deterministic).

- Multinomial Sampling is also faster on the T4 GPU.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install transformers
!pip install datasets
!pip install accelerate -U
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.6 MB/s eta 0:00:00


In [3]:
PATH_PROJECT = "/content/gdrive/MyDrive/projects_work_single/taylor_swift_albums/"
PATH_DATA = "/content/gdrive/MyDrive/projects_work_single/taylor_swift_albums/taylor_swift_data/"
PATH_TRACKS = PATH_DATA + "Tabular/"
PATH_ALBUM = PATH_DATA + "Albums/"

PATH_TO_PROJECT_CODE = "/content/gdrive/MyDrive/projects_work_single/taylor_swift_albums/distilgpt2_taylor_experiments_2023/generator_taylor_swift_lyrics"
PATH_EXPERIMENTS = "/content/gdrive/MyDrive/projects_work_single/taylor_swift_albums/distilgpt2_taylor_experiments_2023/"

In [4]:
import numpy as np
import pandas as pd
import os

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


In [5]:
path_to_saved_model = os.path.join(PATH_EXPERIMENTS, "finetuned_model/")

model = AutoModelForCausalLM.from_pretrained(path_to_saved_model)
tokenizer = AutoTokenizer.from_pretrained(path_to_saved_model)


## Example 1

In [63]:
prompt_verses = "I\'m just gonna shake, shake, shake, shake, shake\nI shake it off, I shake it off (Ha!)\nHeartbreakers gonna break, break, break, break, break (Mmm)"
inputs_prompt_verses = tokenizer(prompt_verses, return_tensors="pt").input_ids

## Using *Multinomial Sampling (stochastic)* with

 - *Top-k most likely tokens*

 - *Top-p for sum probability of tokens*

Note: context window is 1024 tokens. Using a stochastic method, sample several pick the best one.

In [73]:
outputs = model.generate(inputs_prompt_verses, pad_token_id=tokenizer.pad_token_id, max_new_tokens=800, do_sample=True, top_k=30, top_p=0.95)

In [74]:
print("--- first few verses of song ---\n\n")
print(prompt_verses)

--- first few verses of song ---


I'm just gonna shake, shake, shake, shake, shake
I shake it off, I shake it off (Ha!)
Heartbreakers gonna break, break, break, break, break (Mmm)


In [75]:
print("--- song completion using language model (fine-tuned distilgpt2) ---\n\n")
print(tokenizer.decode(outputs[0]))

--- song completion using language model (fine-tuned distilgpt2) ---


I'm just gonna shake, shake, shake, shake, shake
I shake it off, I shake it off (Ha!)
Heartbreakers gonna break, break, break, break, break (Mmm)
The best way to break me is by throwing a ring, throwing the ring over my head
I'm just gonna shake, shake, shake, shake, shake
I'm just gonna shake, shake, shake, shake, shake
I'm just gonna shake, shake, shake, shake, shake

[Outro]
I'm just gonna shake, shake, shake, shake, shake, shake
I'm just gonna shake, shake, shake, shake
[Verse 2]
I wake up screaming, screaming, screaming
But I don't mean you don't, I mean you don't

[Pre-Chorus]
And I wake up screaming, screaming, screaming
But I don't mean you don't, I mean you don't, I mean you don't, I mean you don't
I'm just gonna shake, shake, shake, shake
I'm just gonna shake, shake, shake, shake, shake
I'm just gonna shake, shake, shake, shake, shake
I'm just gonna shake, shake, shake, shake, shake
I'm just gonna shake, s

## Using *Constrastive Search (deterministic)*  with

 - *Top-k most likely tokens*

 - *Penalty α (alpha) for a diverse non-reapting output in long texts*

Note: context window is 1024 tokens. Using a deterministic method, will generate always the same output. Change hyper-parameters for diferent output. Higher α will penalize the log-likelihood of causal model more.

In [76]:
outputs = model.generate(inputs_prompt_verses, pad_token_id=tokenizer.pad_token_id, max_new_tokens=800, top_k=6, penalty_alpha=0.8)

In [77]:
print("--- first few verses of song ---\n\n")
print(prompt_verses)

--- first few verses of song ---


I'm just gonna shake, shake, shake, shake, shake
I shake it off, I shake it off (Ha!)
Heartbreakers gonna break, break, break, break, break (Mmm)


In [78]:
print("--- song completion using language model (fine-tuned distilgpt2) ---\n\n")
print(tokenizer.decode(outputs[0]))

--- song completion using language model (fine-tuned distilgpt2) ---


I'm just gonna shake, shake, shake, shake, shake
I shake it off, I shake it off (Ha!)
Heartbreakers gonna break, break, break, break, break (Mmm)
I shake it off, I shake it off (Ha!)

[Chorus 1]
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't know what to say
I don't k

## Example 2

In [49]:
prompt_verses = "[Intro]\nAh-ah, ah-ah\nAh-ah, ah-ah\n\n[Verse 1]\nYou were in college working part-time, waitin\' tables\nLeft a small town, never looked back\n"
inputs_prompt_verses = tokenizer(prompt_verses, return_tensors="pt").input_ids

## Using *Multinomial Sampling (stochastic)*


In [43]:
outputs = model.generate(inputs_prompt_verses, pad_token_id=tokenizer.pad_token_id, max_new_tokens=800, do_sample=True, top_k=50, top_p=0.95)

In [44]:
print("--- first few verses of song ---\n\n")
print(prompt_verses)

--- first few verses of song ---


[Intro]
Ah-ah, ah-ah
Ah-ah, ah-ah

[Verse 1]
You were in college working part-time, waitin' tables
Left a small town, never looked back



In [45]:
print("--- song completion using language model (fine-tuned distilgpt2) ---\n\n")
print(tokenizer.decode(outputs[0]))

--- song completion using language model (fine-tuned distilgpt2) ---


[Intro]
Ah-ah, ah-ah
Ah-ah, ah-ah

[Verse 1]
You were in college working part-time, waitin' tables
Left a small town, never looked back
A few weeks later, you're still looking back
Now you just need your head in your hands
[Pre-Chorus]
You were on your first date
You said it was the best of times
You're a great man
(And I can say)
Oh, yeah
Yeah, oh, yeah

[Chorus]
Can you imagine me?
And the sky was the limit
So grab your hand and hold me tight
Say it's love with the touch of a hand
Or a kiss and run and grab the kiss
But you couldn't take it so
You're the one that got you there
I'll be the one that got you

[Verse 2]
I think we walked a golden road
But time turns everything around

[Pre-Chorus]
And you got me

[Chorus]
Can you imagine me?
And the sky was the limit
So grab your hand and hold me tight
Say it's love with the touch of a hand
Or a kiss and run and grab the kiss
But you couldn't take it so
You're the one

## Using *Constrastive Search (deterministic)*


In [50]:
outputs = model.generate(inputs_prompt_verses, pad_token_id=tokenizer.pad_token_id, max_new_tokens=800, top_k=15, penalty_alpha=0.9)

In [51]:
print("--- first few verses of song ---\n\n")
print(prompt_verses)

--- first few verses of song ---


[Intro]
Ah-ah, ah-ah
Ah-ah, ah-ah

[Verse 1]
You were in college working part-time, waitin' tables
Left a small town, never looked back



In [52]:
print("--- song completion using language model (fine-tuned distilgpt2) ---\n\n")
print(tokenizer.decode(outputs[0]))

--- song completion using language model (fine-tuned distilgpt2) ---


[Intro]
Ah-ah, ah-ah
Ah-ah, ah-ah

[Verse 1]
You were in college working part-time, waitin' tables
Left a small town, never looked back
And you didn't know what you were thinking
[Preface]
I'm not the type of kind of kind of kind of naive
And I'm not trying to be mean

[Chorus]
Ah-ah, ah-uh ah-ah
Ah-ah, ah-ah
Ah-uh, ah-ah

[Verse 2)
You used to be a little kid
You used to be one of the lucky few
And you used to be a little kid

[Pre-Chorus]
Ah-ah, ah-ah
Ah-ah, ah-ah

[Chorus]
Ah-ah, ah-ah
Ah-ah, ah-ah

[Bridge]
Don't you remember?
Don't you remember?
Don't you remember?
Don't you remember?
Don't you remember?
Don't you remember?
Yeah, yeah

[Chorus 1]
I used to be a little kid
You used to be one of the kind of naive
And I'm not the type of kind of naive
And I'm not the type of kind of naive
And I'm not the type of kind of naive
And I'm not the type of kind of naive
And I'm not the type of kind of naive
And I'm not t